In [1]:
from PIL import Image, ImageOps
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import os, shutil
import time
import scipy
from scipy import ndimage
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import cv2
from keras import models
from keras import layers
from sklearn.utils import class_weight

# Now lets try and to make the results better

In [2]:
# Create a data generator
datagen2 = ImageDataGenerator(
    rescale=1./255, # normalize pixel values
    rotation_range=20, # randomly rotate images in the range
    zoom_range=0.2, # randomly zoom image 
    width_shift_range=0.2, # randomly shift images horizontally
    height_shift_range=0.2, # randomly shift images vertically 
    horizontal_flip=True) # randomly flip images

In [3]:
# Use flow_from_directory to read images from folders
train_generator2 = datagen2.flow_from_directory(
        'CellData/chest_xray/train', # point to the parent directory
        target_size=(299, 299), # resize images to 299x299
        batch_size=32,
        class_mode='binary') # binary for two classes

# Similarly create validation and test generators
val_generator2 = datagen2.flow_from_directory(
        'CellData/chest_xray/val',
        target_size=(299, 299),
        batch_size=32,
        class_mode='binary')

test_generator2 = datagen2.flow_from_directory(
        'CellData/chest_xray/test',
        target_size=(299, 299),
        batch_size=32,
        class_mode='binary')


Found 4098 images belonging to 2 classes.
Found 879 images belonging to 2 classes.
Found 879 images belonging to 2 classes.


In [5]:
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Dropout, BatchNormalization

model2 = models.Sequential()

model2.add(layers.Conv2D(64, (5, 5), padding='same', activation='relu',
                        input_shape=(299 , 299,  3)))
model2.add(BatchNormalization())
model2.add(layers.MaxPooling2D((2, 2)))

model2.add(layers.Conv2D(64, (4, 4), padding='same', activation='relu'))
model2.add(BatchNormalization())
model2.add(layers.MaxPooling2D((2, 2)))
model2.add(Dropout(0.25)) # Added dropout

model2.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
model2.add(BatchNormalization())
model2.add(layers.MaxPooling2D((2, 2)))

model2.add(layers.Flatten())
model2.add(layers.Dense(128, activation='relu'))
model2.add(BatchNormalization())
model2.add(Dropout(0.5)) # Added dropout
model2.add(layers.Dense(256, activation='relu'))
model2.add(BatchNormalization())
model2.add(layers.Dense(1, activation='sigmoid'))

model2.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['acc'])


In [6]:
from collections import Counter
import numpy as np

# Get class distribution
counter = Counter(train_generator2.classes)                          
max_val = float(max(counter.values()))       
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}                     

print(counter)
print(max_val)
print(class_weights)

Counter({1: 2991, 0: 1107})
2991.0
{0: 2.7018970189701896, 1: 1.0}


In [7]:
epochs = 100

steps_per_epoch2 = np.ceil(train_generator2.samples / train_generator2.batch_size)
validation_steps2 = np.ceil(val_generator2.samples / val_generator2.batch_size)

history = model2.fit(
    train_generator2,
    steps_per_epoch=int(steps_per_epoch2),
    epochs=epochs,
    validation_data=val_generator2,
    validation_steps=int(validation_steps2),
    class_weight=class_weights)



Epoch 1/100


2023-07-13 00:04:21.673100: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-07-13 00:04:22.094281: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


129/129 [==============================] - ETA: 0s - loss: 0.6953 - acc: 0.7631

2023-07-13 00:05:24.814866: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


129/129 [==============================] - 76s 545ms/step - loss: 0.6953 - acc: 0.7631 - val_loss: 2.4025 - val_acc: 0.7292
Epoch 2/100
129/129 [==============================] - 75s 573ms/step - loss: 0.5150 - acc: 0.8490 - val_loss: 0.5926 - val_acc: 0.7338
Epoch 3/100
129/129 [==============================] - 71s 542ms/step - loss: 0.4570 - acc: 0.8724 - val_loss: 1.3074 - val_acc: 0.7292
Epoch 4/100
129/129 [==============================] - 71s 543ms/step - loss: 0.4263 - acc: 0.8736 - val_loss: 3.3235 - val_acc: 0.7292
Epoch 5/100
129/129 [==============================] - 72s 553ms/step - loss: 0.4327 - acc: 0.8758 - val_loss: 2.8407 - val_acc: 0.7133
Epoch 6/100
129/129 [==============================] - 70s 541ms/step - loss: 0.4127 - acc: 0.8770 - val_loss: 3.4783 - val_acc: 0.2912
Epoch 7/100
129/129 [==============================] - 71s 547ms/step - loss: 0.4285 - acc: 0.8758 - val_loss: 0.4857 - val_acc: 0.7861
Epoch 8/100
129/129 [==============================] - 74s 5

Epoch 61/100
129/129 [==============================] - 71s 547ms/step - loss: 0.2268 - acc: 0.9407 - val_loss: 0.1814 - val_acc: 0.9408
Epoch 62/100
129/129 [==============================] - 72s 563ms/step - loss: 0.2371 - acc: 0.9370 - val_loss: 1.3704 - val_acc: 0.7315
Epoch 63/100
129/129 [==============================] - 72s 549ms/step - loss: 0.2504 - acc: 0.9339 - val_loss: 2.6244 - val_acc: 0.7292
Epoch 64/100
129/129 [==============================] - 71s 545ms/step - loss: 0.2485 - acc: 0.9300 - val_loss: 0.6436 - val_acc: 0.7304
Epoch 65/100
129/129 [==============================] - 71s 546ms/step - loss: 0.2542 - acc: 0.9351 - val_loss: 1.0928 - val_acc: 0.6143
Epoch 66/100
129/129 [==============================] - 71s 551ms/step - loss: 0.2201 - acc: 0.9407 - val_loss: 0.7847 - val_acc: 0.6940
Epoch 67/100
129/129 [==============================] - 71s 549ms/step - loss: 0.2331 - acc: 0.9383 - val_loss: 0.3585 - val_acc: 0.8487
Epoch 68/100
129/129 [===================

In [8]:
results_train = model2.evaluate(train_generator2)
results_test = model2.evaluate(val_generator2)
print('Training set performance:', results_train)
print('Validation set performance:', results_test)

28/28 [==============================] - 13s 468ms/step - loss: 0.1850 - acc: 0.9283
Training set performance: [0.18054071068763733, 0.9289897680282593]
Validation set performance: [0.18498709797859192, 0.928327739238739]


In [9]:
model2.save('modelv3weightedreg.h5')  # creates a HDF5 file 'my_model.h5'

## tried normalization again, didn't work